In [38]:
!pip install librosa
from IPython import display

from os.path import join

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim

import librosa
import pickle
import numpy as np

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
source_file_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/train/source_all.pickle"
noise_file_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/train/noise_all.pickle"
mixed_file_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/train/mixed_all.pickle"

In [0]:
source_files = []
noise_files = []
mixed_files = []
ibm_list = []

def get_ibm(noise_mat, source_mat):
    #This function calculates ibm mask
    return 1 * (source_mat > noise_mat) 

def pad_or_truncate(tensr):
    shape_zero_dim = tensr.shape[0] 
    if not isinstance(tensr, torch.Tensor):
        tensr = torch.from_numpy(tensr)
    if shape_zero_dim < 100:
        return F.pad(input=tensr, pad=(0,0,0, 100 - shape_zero_dim))
    elif shape_zero_dim > 100:
        return tensr[:100,:]
    else:
        return tensr

def get_val_from_pickle(filename, ftype, append_to_arr):
    with open(filename, 'rb') as f:
        dict_file = pickle.load(f)
        for i in range(len(dict_file)):
            if i<10:
                wav = dict_file[ftype+"000"+str(i)+".wav"]['stft_abs']
                append_to_arr.append(pad_or_truncate(wav.T))
            elif i>9 and i<100:
                wav = dict_file[ftype+"00"+str(i)+".wav"]['stft_abs']
                append_to_arr.append(pad_or_truncate(wav.T))
            elif i>99 and i<1000:
                wav = dict_file[ftype+"0"+str(i)+".wav"]['stft_abs']
                append_to_arr.append(pad_or_truncate(wav.T))
            else:
                wav = dict_file[ftype+""+str(i)+".wav"]['stft_abs']
                append_to_arr.append(pad_or_truncate(wav.T))
    return append_to_arr

def get_sftf_from_pickle(filename, ftype, append_to_arr):
    with open(filename, 'rb') as f:
        dict_file = pickle.load(f)
        for i in range(len(dict_file)):
            if i<10:
                wav = dict_file[ftype+"000"+str(i)+".wav"]['stft']
                append_to_arr.append(wav.T)
            elif i>9 and i<100:
                wav = dict_file[ftype+"00"+str(i)+".wav"]['stft']
                append_to_arr.append(wav.T)
            elif i>99 and i<1000:
                wav = dict_file[ftype+"0"+str(i)+".wav"]['stft']
                append_to_arr.append(wav.T)
            else:
                wav = dict_file[ftype+""+str(i)+".wav"]['stft']
                append_to_arr.append(wav.T)
    return append_to_arr

source_files = get_val_from_pickle(source_file_pickle, 'trs', source_files)
noise_files = get_val_from_pickle(noise_file_pickle, 'trn', noise_files)
mixed_files = get_val_from_pickle(mixed_file_pickle, 'trx', mixed_files)

for i in range(len(noise_files)):
    ibm_list.append(
        get_ibm(noise_files[i], source_files[i])
    )


I have created the stft and their abs value of those. Stored them in a dictionary and pickled the dictonary and uploaded it on my drive.

This part of code takes care of preprocessing which involves, padding or truncating. Calculations of ibm masks etc.

In [42]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm1  = nn.LSTM(513, 513, batch_first=True)
        self.lstm2  = nn.LSTM(513, 513, batch_first=True)
        self.fc = nn.Linear(513,513)

    def forward(self, x):
        op, _ = self.lstm1(x)
        op, _ = self.lstm2(op)
        op2 = F.sigmoid(self.fc(op))
        return op2

net = Net()
print(net)
learning_rate = 1e-3
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

Net(
  (lstm1): LSTM(513, 513, batch_first=True)
  (lstm2): LSTM(513, 513, batch_first=True)
  (fc): Linear(in_features=513, out_features=513, bias=True)
)


In [0]:
class AudioDataset(Dataset):
    def __init__(self, data, label):
        self.sample = torch.stack(data).float()
        self.label = torch.stack(label).float()
        
    def __len__(self):
        if len(self.sample) == len(self.label):
            return len(self.sample)
    
    def __getitem__(self, idx):
        return self.sample[idx], self.label[idx]

ds = AudioDataset(mixed_files, ibm_list)
train_loader = DataLoader(ds, batch_size=400, shuffle=False, num_workers=2)



In [44]:
epoch_count = 50
for epoch in range(epoch_count):
    running_loss = 0.0
    for index, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad() 

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if index % 3 == 2:
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, index + 1, running_loss / 3))
            running_loss = 0.0

print("Done training")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     3] loss: 0.248992
[2,     3] loss: 0.247087
[3,     3] loss: 0.239145
[4,     3] loss: 0.238331
[5,     3] loss: 0.234305
[6,     3] loss: 0.231520
[7,     3] loss: 0.225339
[8,     3] loss: 0.222450
[9,     3] loss: 0.215089
[10,     3] loss: 0.210606
[11,     3] loss: 0.205771
[12,     3] loss: 0.198151
[13,     3] loss: 0.194126
[14,     3] loss: 0.190219
[15,     3] loss: 0.186370
[16,     3] loss: 0.182570
[17,     3] loss: 0.179893
[18,     3] loss: 0.176964
[19,     3] loss: 0.174926
[20,     3] loss: 0.170423
[21,     3] loss: 0.167864
[22,     3] loss: 0.165541
[23,     3] loss: 0.162556
[24,     3] loss: 0.160261
[25,     3] loss: 0.160502
[26,     3] loss: 0.158927
[27,     3] loss: 0.157614
[28,     3] loss: 0.153914
[29,     3] loss: 0.153137
[30,     3] loss: 0.151212
[31,     3] loss: 0.148266
[32,     3] loss: 0.146727
[33,     3] loss: 0.146079
[34,     3] loss: 0.146575
[35,     3] loss: 0.144845
[36,     3] loss: 0.142155
[37,     3] loss: 0.141583
[38,     3

In [0]:
valnoise_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/validation/noise_val.pickle"
valsource_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/validation/source_val.pickle"
valmixed_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/validation/mix_val.pickle"

In [46]:
#abs files
source_val_files = []
noise_val_files = []
mixed_val_files = []
#complex files
source_stft_files = []
noise_stft_files = []
mixed_stft_files = []
source_val_files = get_val_from_pickle(valsource_pickle, 'vs', source_val_files)
noise_val_files = get_val_from_pickle(valnoise_pickle, 'vn', noise_val_files)
mixed_val_files = get_val_from_pickle(valmixed_pickle, 'vx', mixed_val_files)

source_stft_files = get_sftf_from_pickle(valsource_pickle, 'vs', source_stft_files)
noise_stft_files = get_sftf_from_pickle(valnoise_pickle, 'vn', noise_stft_files)
mixed_stft_files = get_sftf_from_pickle(valmixed_pickle, 'vx', mixed_stft_files)

val_ibm = []

for i in range(len(noise_val_files)):
    val_ibm.append(
        get_ibm(noise_val_files[i], source_val_files[i])
    )
print(
    len(source_val_files), len(noise_val_files), len(mixed_val_files), len(val_ibm)
)

1200 1200 1200 1200


In [0]:
ds = AudioDataset(mixed_val_files, val_ibm)
val_loader = DataLoader(ds, batch_size=512, shuffle=False)

def calculate_audio(mask, noise):
    mask_size = 100
    noise_size = 100
    if mask.shape[0] < noise.shape[0]:
        return mask.data.cpu().numpy()*noise[:mask.shape[0],:] 
    else:
        return mask[:noise.shape[0],:].data.cpu().numpy()*noise

def getsnr(t, t_bar):
    snr = np.dot(t,t.T)/np.dot(
            (t[:len(t_bar)]-t_bar),
            (t[:len(t_bar)]-t_bar).T
    )
    print("SNR is {}".format(snr))
    return snr
    

In [48]:
cross_validation_snr_array = []
with torch.no_grad():
    for index, data in enumerate(val_loader):
        inputs, labels = data
        optimizer.zero_grad() 
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        for index2, op in enumerate(outputs):
            op_mask = 1 * (op > 0.5)
            lsss = calculate_audio(op_mask, mixed_stft_files[index2])

            pred_istft = librosa.core.istft(lsss, hop_length=513)
            original_sound = librosa.core.istft(
                source_stft_files[index2], hop_length=513
            )
            snr_stuff = getsnr(original_sound, pred_istft) 
            print("snr for item {} is {}".format(index2, snr_stuff))
            cross_validation_snr_array.append(snr_stuff)
        print(
            '[%d, %5d] loss: %.6f' %(
                epoch + 1, index + 1, loss.item()
            )
        )


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


SNR is 63.20085525512695
snr for item 0 is 63.20085525512695
SNR is 6.36367130279541
snr for item 1 is 6.36367130279541
SNR is 7.17567253112793
snr for item 2 is 7.17567253112793
SNR is 1.4942082166671753
snr for item 3 is 1.4942082166671753
SNR is 1.4511268138885498
snr for item 4 is 1.4511268138885498
SNR is 4.968997478485107
snr for item 5 is 4.968997478485107
SNR is 2.3781862258911133
snr for item 6 is 2.3781862258911133
SNR is 6.061437129974365
snr for item 7 is 6.061437129974365
SNR is 2.4517412185668945
snr for item 8 is 2.4517412185668945
SNR is 1.4178576469421387
snr for item 9 is 1.4178576469421387
SNR is 0.6652364730834961
snr for item 10 is 0.6652364730834961
SNR is 0.781225323677063
snr for item 11 is 0.781225323677063
SNR is 0.6769945025444031
snr for item 12 is 0.6769945025444031
SNR is 0.6595921516418457
snr for item 13 is 0.6595921516418457
SNR is 0.8330897092819214
snr for item 14 is 0.8330897092819214
SNR is 0.6643646955490112
snr for item 15 is 0.6643646955490112
SN

In [49]:
print(np.average(cross_validation_snr_array))

3.1535788


Here we can see that the average snr over validation files is positive.

In [50]:
testmixed_pickle = "/content/drive/My Drive/Deep Learning Systems/Assignment3Part2Data/test/test_mix.pickle"

#abs files
mixed_test_abs = []
mixed_test_stft = []
mixed_test_abs = get_val_from_pickle(testmixed_pickle, 'tex', mixed_test_abs)
mixed_test_stft = get_sftf_from_pickle(testmixed_pickle, 'tex', mixed_test_stft)

test_ibm = []
#generate this 
generated_test_istft = []
with torch.no_grad():
    for index, ip in enumerate(mixed_test_abs):
        outputs = net(ip.unsqueeze(0))
        lsss = calculate_audio(op_mask, mixed_test_stft[index])
        pred_istft = librosa.core.istft(lsss, hop_length=513)
        generated_test_istft.append(pred_istft)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [51]:
print(len(generated_test_istft))

400


In [0]:
sr_for_generated_test_istft=[]
def get_sr_for_test_audio(filename, ftype, append_to_arr):
    with open(filename, 'rb') as f:
        dict_file = pickle.load(f)
        for i in range(len(dict_file)):
            if i<10:
                append_to_arr.append(
                    dict_file[ftype+"000"+str(i)+".wav"]['sr']
                )
                
            elif i>9 and i<100:
                append_to_arr.append(
                    dict_file[ftype+"00"+str(i)+".wav"]['sr']
                )
            elif i>99 and i<1000:
                append_to_arr.append(
                    dict_file[ftype+"0"+str(i)+".wav"]['sr']
                )
            else:
                append_to_arr.append(
                    dict_file[ftype+""+str(i)+".wav"]['sr']
                )
    return append_to_arr
#We can use following line of code to generate test output
# sr_for_generated_test_istft = get_sr_for_test_audio(testmixed_pickle, 'tex', sr_for_generated_test_istft)

# display.display(display.Audio(
#     data=generated_test_istft[id], rate=sr_for_generated_test_istft[id])
# )